In [1]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 551.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 2.1 MB/s eta 0:00:00a 0:00:01


In [2]:
!pip install spacy[cuda110]

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
tensorflow 2.6.4 requires h5py~=3.1.0, but you have h5py 3.7.0 which is incompatible.
tensorflow 2.6.4 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
tensorflow 2.6.4 requires tensorboard<2.7,>=2.6.0, but you have tensorboard 2.10.0 which is incompatible.
tensorflow 2.6.4 requires typing-extensions<3.11,>=3.7, but you have typing-extensions 4.1.1 which is incompatible.
tensorflow-transform 1.9.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,<2.10,>=1.15.5, but y

In [3]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 27.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:

import numpy as np
import pandas as pd 
import re
import contractions
import os,sys
import string
import spacy
from nltk.tokenize import word_tokenize

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/million-headlines/abcnews-date-text.csv
/kaggle/input/india-headlines-news-dataset/india-news-headlines.csv


In [5]:
df1 = pd.read_csv('../input/million-headlines/abcnews-date-text.csv')
df2 = pd.read_csv('../input/india-headlines-news-dataset/india-news-headlines.csv')

In [6]:
df1.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [7]:
df2.head()

,publish_date,headline_category,headline_text
0,20010102,unknown,Status quo will not be disturbed at Ayodhya; s...
1,20010102,unknown,Fissures in Hurriyat over Pak visit
2,20010102,unknown,America's unwanted heading for India?
3,20010102,unknown,For bigwigs; it is destination Goa
4,20010102,unknown,Extra buses to clear tourist traffic


In [8]:
df1.shape

(1244184, 2)

In [9]:
df2.shape

(3650970, 3)

In [10]:
df1.describe()

,publish_date
count,1.244184e+06
mean,2.011021e+07
std,4.841151e+04
min,2.003022e+07
25%,2.007062e+07
50%,2.011062e+07
75%,2.015022e+07
max,2.021123e+07


In [11]:
df2.describe()

,publish_date
count,3.650970e+06
mean,2.013193e+07
std,5.252282e+04
min,2.001010e+07
25%,2.010042e+07
50%,2.014023e+07
75%,2.017093e+07
max,2.022033e+07


In [12]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1244184 entries, 0 to 1244183
Data columns (total 2 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   publish_date   1244184 non-null  int64 
 1   headline_text  1244184 non-null  object
dtypes: int64(1), object(1)
memory usage: 19.0+ MB


In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3650970 entries, 0 to 3650969
Data columns (total 3 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   publish_date       int64 
 1   headline_category  object
 2   headline_text      object
dtypes: int64(1), object(2)
memory usage: 83.6+ MB


In [14]:
df1.isnull().sum()

publish_date     0
headline_text    0
dtype: int64

In [15]:
df2.isnull().sum()

publish_date         0
headline_category    0
headline_text        0
dtype: int64

In [16]:
df1.drop('publish_date',axis=1,inplace=True)
df2.drop('publish_date',axis=1,inplace=True)
df2.drop('headline_category',axis=1,inplace=True)

In [17]:
df = pd.concat([df1,df2])

In [18]:
df.shape

(4895154, 1)

In [19]:
df['headline_text'].unique()

array(['aba decides against community broadcasting licence',
       'act fire witnesses must be aware of defamation',
       'a g calls for infrastructure protection summit', ...,
       'Schools organise vax camps for kids',
       'J&K sacks 2 cops; 3 other employees over terror links',
       "Ranbir Kapoor says 'Rishi Kapoor enjoyed his alcohol; food and ensured to have at least one meal together with family'"],
      dtype=object)

In [20]:
for i,t in enumerate(df['headline_text'][-200:-100]):
    print(f'headline {i} \n {t}')

headline 0 
 Vadher stars in SPCT victory
headline 1 
 Five flights from SVPI delayed
headline 2 
 Petrol prices now 101.49/litre
headline 3 
 50 ceramic makers in Morbi stop production
headline 4 
 GCCI opposes staggering by GUVNL
headline 5 
 Booze seized during Covid low destroyed
headline 6 
 Doctors condemn factors that led to ob-gyn's suicide in Rajasthan
headline 7 
 Notify GIDC estates to end double taxation
headline 8 
 ICG ship rescues 9 TN fishermen
headline 9 
 President Ram Nath Kovind; PM Modi; Amit Shah to visit Gujarat in April
headline 10 
 Andhra Pradesh's central varsities to opt for common entrance test
headline 11 
 Public role vital for good SS rank
headline 12 
 J'khand govt nod to Sarhul; Ram Navami processions
headline 13 
 Visakhapatnam: 56 tribal youngsters clear SSC exam
headline 14 
 Maoists call bandh against Lalu conviction; central govt policies
headline 15 
 Rasheed hit 2nd consecutive ton
headline 16 
 Three accused in Nanhe Khan murder case held
headl

In [21]:
df['headline_text']=df['headline_text'].apply(lambda x:contractions.fix(x))

In [22]:
df['headline_text']=df['headline_text'].apply(lambda x : x.lower())

In [23]:
df['headline_text']=df['headline_text'].apply(lambda x : re.sub("\w*\d\w*","",x))

In [24]:
df['headline_text']=df['headline_text'].apply(lambda x : re.sub('[%s]'%re.escape(string.punctuation),'',x))

In [25]:
df['headline_text']=df['headline_text'].apply(lambda x: re.sub(' +',' ',x))

In [26]:
spacy.require_gpu()
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])
df['lemmatized'] = df['headline_text'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))

In [27]:
!touch data.csv

In [28]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [29]:
df.drop('headline_text',axis=1,inplace=True)

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df['tokenized']=df['lemmatized'].apply(lambda x : word_tokenize(str(x)))